In [1]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# !git clone https://github.com/SeanNaren/warp-ctc.git
# !cd warp-ctc; mkdir build; cd build; cmake ..;make
# !cd warp-ctc/pytorch_binding;python setup.py install


In [0]:
import warpctc_pytorch

In [0]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks")
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [0]:
import random
import torch
from torch.utils.data import Dataset
from torch.utils.data import sampler
import torchvision.transforms as transforms
import lmdb
import six
import sys
from PIL import Image
import numpy as np
import os
import json
import cv2


class importDataset(Dataset):

    def __init__(self, root=None, label_dict = None, transform=None, target_transform=None):
        self.key = os.listdir(root)
        self.root = root
        nSamples = len(os.listdir(root))
        self.nSamples = nSamples

        self.transform = transform
        self.target_transform = target_transform
        with open(label_dict, 'r') as j:
            self.label_dict = json.loads(j.read())
        

    def __len__(self):
        return self.nSamples
    
    def get_label(self):
          return self.label_dict

    def get_dir(self):
        return self.dir

    def get_key(self):
        return self.key



    def __getitem__(self, index):
        # print("Index", index)
        # print("Dataset", len(self))
        assert index <= len(self), 'index range error'

        # index += 1


        img = cv2.imread(os.path.join(self.root, self.key[index]), cv2.IMREAD_GRAYSCALE)
        label = self.label_dict[self.key[index]]
        
        if self.transform is not None:
            img = self.transform(img)


        if self.target_transform is not None:
            label = self.target_transform(label)
        

        return (img, label)
        

class resizeNormalize(object):

    def __init__(self, size):
        self.size = size
        self.toTensor = transforms.ToTensor()

    def __call__(self, img):
        img = cv2.resize(img, self.size)
        img = self.toTensor(img)
        img.sub_(0.5).div_(0.5)
        return img


class randomSequentialSampler(sampler.Sampler):

    def __init__(self, data_source, batch_size):
        self.num_samples = len(data_source)
        self.batch_size = batch_size

    def __iter__(self):
        n_batch = len(self) // self.batch_size
        tail = len(self) % self.batch_size
        index = torch.LongTensor(len(self)).fill_(0)
        for i in range(n_batch):
            random_start = random.randint(0, len(self) - self.batch_size)
            # batch_index = random_start + torch.arange(0, self.batch_size - 1)
            batch_index = random_start + torch.arange(0, self.batch_size)
            index[i * self.batch_size:(i + 1) * self.batch_size] = batch_index
        # deal with tail
        if tail:
            random_start = random.randint(0, len(self) - self.batch_size)
            # tail_index = random_start + torch.arange(0, tail - 1)
            tail_index = random_start + torch.arange(0, tail)
            index[(i + 1) * self.batch_size:] = tail_index

        return iter(index)

    def __len__(self):
        return self.num_samples


class alignCollate(object):

    def __init__(self, imgH=32, imgW=100, keep_ratio=False, min_ratio=1):
        self.imgH = imgH
        self.imgW = imgW
        self.keep_ratio = keep_ratio
        self.min_ratio = min_ratio

    def __call__(self, batch):
        images, labels = zip(*batch)
        

        imgH = self.imgH
        imgW = self.imgW
        if self.keep_ratio:
            ratios = []
            for image in images:
                # print(type(image))
                w, h = image.shape[1], image.shape[0]
                ratios.append(w / float(h))
            ratios.sort()
            max_ratio = ratios[-1]
            imgW = int(np.floor(max_ratio * imgH))
            imgW = max(imgH * self.min_ratio, imgW)  # assure imgH >= imgW

        transform = resizeNormalize((imgW, imgH))
        images = [transform(image) for image in images]
        images = torch.cat([t.unsqueeze(0) for t in images], 0)

          
        return images, labels


In [20]:
from __future__ import print_function
from __future__ import division

import argparse
import random
import torch
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
import numpy as np
from warpctc_pytorch import CTCLoss
import os
from crnn_pytorch import utils
# import new_dataset

from crnn_pytorch.models.crnn import CRNN



trainroot ="datasets/icdar2015/recognition_train"
trainlabel= "datasets/icdar2015/train_labels_2.json"
valroot = "datasets/icdar2015/recognition_test"
vallabel = "datasets/icdar2015/test_labels_2.json"
cuda= True

workers = 2

batchSize = 64
imgH = 32
imgW = 100
nh = 256
nepoch = 5

ngpu = 1

# pretrained = "expr/netCRNN_24.pth"
pretrained = ""
# alphabet = '#$%&0123456789@ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
alphabet = ' ,.0123456789ABCDGHIKLMNOPQRSTUVXY\\abcdefghijklmnopqrstuvwxyÀÁÂÃÒÓÔÙÚÝàáâãèéêìíòóôõùúýăĐđĨĩŨũƠơƯưẠạẢảấẦầẩẫẬậắặẹẽẾếềỂểễỆệỉỊịọỏốồỔổỗỘộỚớỜờỞởỡợỤụỦủứừửữỰựỳỷỹ'
expr_dir = "expr_viet"
displayInterval = 30
n_test_disp = 10
valInterval = 180
saveInterval = 180
lr = 0.01
beta1 = 0.5

adam = False
adadelta = True
keep_ratio = True
manualSeed = 1234
random_sample = True


if not os.path.exists(expr_dir):
    os.makedirs(expr_dir)

random.seed(manualSeed)
np.random.seed(manualSeed)
torch.manual_seed(manualSeed)

cudnn.benchmark = True

if torch.cuda.is_available() and not cuda:
    print("WARNING: You have a CUDA device, so you should probably run with --cuda")

#TODO
train_dataset = importDataset(root=trainroot, label_dict = trainlabel)
assert train_dataset
if not random_sample:
    sampler = randomSequentialSampler(train_dataset, batchSize)
else:
    sampler = None
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batchSize,
    # shuffle=True, sampler=sampler,
    shuffle=True, sampler=None,
    # shuffle = False, sampler = randomSequentialSampler(train_dataset, batchSize),
    num_workers=int(workers),
    collate_fn= alignCollate(imgH=imgH, imgW=imgW, keep_ratio=keep_ratio))
test_dataset = importDataset(
    root=valroot, label_dict = vallabel, transform= resizeNormalize((100, 32)))

nclass = len(alphabet) + 1
nc = 1

converter = utils.strLabelConverter(alphabet, ignore_case=False)
criterion = CTCLoss()


# custom weights initialization called on crnn
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


crnn = CRNN(imgH, nc, nclass, nh)
crnn.apply(weights_init)

print(crnn)

image = torch.FloatTensor(batchSize, 3, imgH, imgH)
text = torch.IntTensor(batchSize * 5)
length = torch.IntTensor(batchSize)

if cuda== True:
    crnn.cuda()
    crnn = torch.nn.DataParallel(crnn, device_ids=range(ngpu))
    if pretrained != '':
      print('loading pretrained model from %s' % pretrained)
      crnn.load_state_dict(torch.load(pretrained))
    image = image.cuda()
    criterion = criterion.cuda()

image = Variable(image)
text = Variable(text)
length = Variable(length)

# loss averager
loss_avg = utils.averager()

# setup optimizer
if adam:
    optimizer = optim.Adam(crnn.parameters(), lr=lr,
                           betas=(beta1, 0.999))
elif adadelta:
    optimizer = optim.Adadelta(crnn.parameters())
else:
    optimizer = optim.RMSprop(crnn.parameters(), lr=lr)



CRNN(
  (cnn): Sequential(
    (conv0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu0): ReLU(inplace=True)
    (pooling0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu1): ReLU(inplace=True)
    (pooling1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batchnorm2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (conv3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu3): ReLU(inplace=True)
    (pooling2): MaxPool2d(kernel_size=(2, 2), stride=(2, 1), padding=(0, 1), dilation=1, ceil_mode=False)
    (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batchnorm4): BatchNorm2d(512, eps=1e-05, momentum=0.

In [0]:
len('0123456789abcdefghijklmnopqrstuvwxyz')

36

In [6]:
crnn_2 = CRNN(imgH, nc, 37, nh)
crnn = CRNN(imgH, nc, nclass, nh)
crnn.apply(weights_init)

CRNN(
  (cnn): Sequential(
    (conv0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu0): ReLU(inplace=True)
    (pooling0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu1): ReLU(inplace=True)
    (pooling1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batchnorm2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (conv3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu3): ReLU(inplace=True)
    (pooling2): MaxPool2d(kernel_size=(2, 2), stride=(2, 1), padding=(0, 1), dilation=1, ceil_mode=False)
    (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batchnorm4): BatchNorm2d(512, eps=1e-05, momentum=0.

In [0]:
# crnn_2.cuda()
# crnn_2 = torch.nn.DataParallel(crnn_2, device_ids=range(ngpu))

In [12]:
crnn_2.load_state_dict(torch.load('crnn_pytorch/data/crnn.pth'))

<All keys matched successfully>

In [13]:
crnn_2.state_dict().keys()

odict_keys(['cnn.conv0.weight', 'cnn.conv0.bias', 'cnn.conv1.weight', 'cnn.conv1.bias', 'cnn.conv2.weight', 'cnn.conv2.bias', 'cnn.batchnorm2.weight', 'cnn.batchnorm2.bias', 'cnn.batchnorm2.running_mean', 'cnn.batchnorm2.running_var', 'cnn.batchnorm2.num_batches_tracked', 'cnn.conv3.weight', 'cnn.conv3.bias', 'cnn.conv4.weight', 'cnn.conv4.bias', 'cnn.batchnorm4.weight', 'cnn.batchnorm4.bias', 'cnn.batchnorm4.running_mean', 'cnn.batchnorm4.running_var', 'cnn.batchnorm4.num_batches_tracked', 'cnn.conv5.weight', 'cnn.conv5.bias', 'cnn.conv6.weight', 'cnn.conv6.bias', 'cnn.batchnorm6.weight', 'cnn.batchnorm6.bias', 'cnn.batchnorm6.running_mean', 'cnn.batchnorm6.running_var', 'cnn.batchnorm6.num_batches_tracked', 'rnn.0.rnn.weight_ih_l0', 'rnn.0.rnn.weight_hh_l0', 'rnn.0.rnn.bias_ih_l0', 'rnn.0.rnn.bias_hh_l0', 'rnn.0.rnn.weight_ih_l0_reverse', 'rnn.0.rnn.weight_hh_l0_reverse', 'rnn.0.rnn.bias_ih_l0_reverse', 'rnn.0.rnn.bias_hh_l0_reverse', 'rnn.0.embedding.weight', 'rnn.0.embedding.bias',

In [14]:
crnn.state_dict().keys()

odict_keys(['cnn.conv0.weight', 'cnn.conv0.bias', 'cnn.conv1.weight', 'cnn.conv1.bias', 'cnn.conv2.weight', 'cnn.conv2.bias', 'cnn.batchnorm2.weight', 'cnn.batchnorm2.bias', 'cnn.batchnorm2.running_mean', 'cnn.batchnorm2.running_var', 'cnn.batchnorm2.num_batches_tracked', 'cnn.conv3.weight', 'cnn.conv3.bias', 'cnn.conv4.weight', 'cnn.conv4.bias', 'cnn.batchnorm4.weight', 'cnn.batchnorm4.bias', 'cnn.batchnorm4.running_mean', 'cnn.batchnorm4.running_var', 'cnn.batchnorm4.num_batches_tracked', 'cnn.conv5.weight', 'cnn.conv5.bias', 'cnn.conv6.weight', 'cnn.conv6.bias', 'cnn.batchnorm6.weight', 'cnn.batchnorm6.bias', 'cnn.batchnorm6.running_mean', 'cnn.batchnorm6.running_var', 'cnn.batchnorm6.num_batches_tracked', 'rnn.0.rnn.weight_ih_l0', 'rnn.0.rnn.weight_hh_l0', 'rnn.0.rnn.bias_ih_l0', 'rnn.0.rnn.bias_hh_l0', 'rnn.0.rnn.weight_ih_l0_reverse', 'rnn.0.rnn.weight_hh_l0_reverse', 'rnn.0.rnn.bias_ih_l0_reverse', 'rnn.0.rnn.bias_hh_l0_reverse', 'rnn.0.embedding.weight', 'rnn.0.embedding.bias',

In [0]:
def removekey(d, keys):
    r = dict(d)
    for key in keys:
      del r[key]
    return r

In [0]:
pretrained_dict = removekey(crnn_2.state_dict(), ('rnn.1.embedding.weight', 'rnn.1.embedding.bias'))

In [0]:
for key in pretrained_dict:
  crnn.state_dict()[key] = pretrained_dict[key]

In [0]:
# crnn_2.state_dict()['cnn.batchnorm6.running_var']

In [0]:
crnn.cuda()
crnn = torch.nn.DataParallel(crnn, device_ids=range(ngpu))
torch.save(crnn.state_dict(), '{0}/netCRNN_viet.pth'.format(expr_dir))

In [0]:
def val(net, dataset, criterion, max_iter=100):
    print('Start val')

    for p in crnn.parameters():
      p.requires_grad = False

    net.eval()
    data_loader = torch.utils.data.DataLoader(
        dataset, shuffle=False, batch_size=batchSize, num_workers=int(workers))
    val_iter = iter(data_loader)

    i = 0
    n_correct = 0
    loss_avg = utils.averager()

    max_iter = min(max_iter, len(data_loader))
    for i in range(max_iter):
      
      data = val_iter.next()
      i += 1
      cpu_images, cpu_texts = data
      batch_size = cpu_images.size(0)
      utils.loadData(image, cpu_images)
      t, l = converter.encode(cpu_texts)
      utils.loadData(text, t)
      utils.loadData(length, l)

      preds = crnn(image)

      preds_size = Variable(torch.IntTensor([preds.size(0)] * batch_size))
      cost = criterion(preds, text, preds_size, length) / batch_size
      loss_avg.add(cost)

      _, preds = preds.max(2,  keepdim=True)
      preds = preds.squeeze(2)
      preds = preds.transpose(1, 0).contiguous().view(-1)
      sim_preds = converter.decode(preds.data, preds_size.data, raw=False)
      for pred, target in zip(sim_preds, cpu_texts):
          if pred == target:
              n_correct += 1

    raw_preds = converter.decode(preds.data, preds_size.data, raw=True)[:n_test_disp]
    for raw_pred, pred, gt in zip(raw_preds, sim_preds, cpu_texts):
        print('%-20s => %-20s, gt: %-20s' % (raw_pred, pred, gt))

    accuracy = n_correct / float(max_iter * batchSize)
    print('Test loss: %f, accuracy: %f' % (loss_avg.val(), accuracy))


def trainBatch(net, criterion, optimizer):
    data = train_iter.next()
    cpu_images, cpu_texts = data

    batch_size = cpu_images.size(0)
    utils.loadData(image, cpu_images)
    t, l = converter.encode(cpu_texts)
    utils.loadData(text, t)
    utils.loadData(length, l)

    preds = crnn(image)
    preds_size = Variable(torch.IntTensor([preds.size(0)] * batch_size))
    cost = criterion(preds, text, preds_size, length) / batch_size
    crnn.zero_grad()
    cost.backward()
    optimizer.step()
    return cost

In [0]:
len("#------------------------e")

26

In [0]:
val(crnn, test_dataset, criterion)

Start val
#------------------------e => #e                  , gt: SALON               
#-------------------#----# => ###                 , gt: ###                 
#--------------------#---# => ###                 , gt: THE                 
#-#----------------------# => ###                 , gt: RESTAURANT          
#--------------------#---# => ###                 , gt: PAUL                
#------------------#-----# => ###                 , gt: BOOKS               
S-a--------------------ane => Saane               , gt: Kinokuniya          
#-#----------------------# => ###                 , gt: Knowledge           
#--------------------#---# => ###                 , gt: ###                 
#-------------------#----# => ###                 , gt: ###                 
Test loss: 9.468168, accuracy: 0.593559


In [0]:
for epoch in range(nepoch):

    train_iter = iter(train_loader)
    i = 0
    while i < len(train_loader):
        for p in crnn.parameters():
            p.requires_grad = True
        crnn.train()

        cost = trainBatch(crnn, criterion, optimizer)
        loss_avg.add(cost)
        i += 1

        if i % displayInterval == 0:
            print('[%d/%d][%d/%d] Loss: %f' %
                  (epoch, nepoch, i, len(train_loader), loss_avg.val()))
            loss_avg.reset()

        if i % valInterval == 0:
            val(crnn, test_dataset, criterion)

        # do checkpointing
        if i % saveInterval == 0:
            torch.save(
                

                crnn.state_dict(), '{0}/netCRNN_{1}_{2}.pth'.format(expr_dir, epoch, i))
        torch.save(
                crnn.state_dict(), '{0}/netCRNN_{1}.pth'.format(expr_dir, epoch))

[0/5][30/186] Loss: 24.486940
[0/5][60/186] Loss: 13.591163
[0/5][90/186] Loss: 12.468540
[0/5][120/186] Loss: 12.258647
[0/5][150/186] Loss: 12.250970
[0/5][180/186] Loss: 12.115555
[1/5][30/186] Loss: 11.295891
[1/5][60/186] Loss: 12.279264
[1/5][90/186] Loss: 11.611892
[1/5][120/186] Loss: 12.013558
[1/5][150/186] Loss: 11.502882
[1/5][180/186] Loss: 11.617794
[2/5][30/186] Loss: 11.327031


KeyboardInterrupt: ignored

In [0]:
original_label = ' !\"#$%&\\\'()+-./0123456789:;=?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[]abcdefghijklmnopqrstuvwxyz´Éé'

In [0]:
valu = list(diction.values())
valu2 = list(diction_2.values())

In [0]:
label_set = set()
for value in valu:
  for char in value:
    if char not in label_set:
      label_set.add(char)
for value in valu2:
  for char in value:
    if char not in label_set:
      label_set.add(char)

In [0]:
"".join(sorted(label_set))

'#$%&0123456789@ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'

In [0]:
remove_dict = [ "!",  "\"", "'", "(", ")", "+", "-", ".", "/", "´", "[", "]", "(", ")", ":", ";", "=" , " ", "?"]
replace_dict = ["É"]


In [0]:
def filter_label(dict): 
  for key in dict.keys():
    for char in remove_dict:
      if char in dict[key]:

        dict[key] = dict[key].replace(char, "")
      for char in replace_dict:
        dict[key] = dict[key].replace(char, "E")
  return dict

In [0]:
test_dict = filter_label(diction)
train_dict = filter_label(diction_2)

In [0]:
# train_dict

In [0]:
with open("datasets/icdar2015/test_labels_2.json", 'w') as fp:
  json.dump(test_dict, fp)
with open("datasets/icdar2015/train_labels_2.json", 'w') as fp:
  json.dump(train_dict, fp)

In [0]:
label_set_2 = set()
for value in list(test_dict.values()):
  for char in value:

    if char not in label_set_2:
      label_set_2.add(char)
for value in list(train_dict.values()):
  for char in value:

    if char not in label_set_2:
      label_set_2.add(char)

In [0]:
# label_set_2

In [0]:
"".join(sorted(label_set_2))

'#$%&0123456789@ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'

In [0]:
label_set = set()
for value in diction.values():
  for char in value:
    if char not in label_set:
      label_set.add(char)

In [0]:
!python crnn_pytorch/new_train.py --adadelta --trainroot "datasets/icdar2015/recognition_train"  \
--trainlabel "datasets/icdar2015/train_labels_2.json" \
--valroot "datasets/icdar2015/recognition_test" --vallabel "datasets/icdar2015/test_labels_2.json"  --cuda \
--alphabet '#$%&0123456789@ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz' \
--displayInterval 30 --valInterval 180 --saveInterval 100 --lr 0.01 --pretrained "expr/netCRNN.pth" --nepoch 25 \
--imgH 32 --imgW 100 --n_test_disp 10

Namespace(adadelta=True, adam=False, alphabet='#$%&0123456789@ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz', batchSize=64, beta1=0.5, cuda=True, displayInterval=30, expr_dir='expr', imgH=32, imgW=100, keep_ratio=False, lr=0.01, manualSeed=1234, n_test_disp=10, nepoch=25, ngpu=1, nh=256, pretrained='expr/netCRNN.pth', random_sample=False, saveInterval=100, trainlabel='datasets/icdar2015/train_labels_2.json', trainroot='datasets/icdar2015/recognition_train', valInterval=180, vallabel='datasets/icdar2015/test_labels_2.json', valroot='datasets/icdar2015/recognition_test', workers=2)
CRNN(
  (cnn): Sequential(
    (conv0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu0): ReLU(inplace=True)
    (pooling0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu1): ReLU(inplace=True)
    (pooling1): MaxPool2d(kernel_size=2, stride=2, padding=0, dila